<a href="https://colab.research.google.com/github/Hassan7838/ai-cold-email-generator/blob/main/ai-cold-email-generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# AI-Powered Cold Email Generator for Job and Client Outreach

# Installing Necessary Packages
!pip install langchain chromadb llama-cpp-python sentence-transformers -U langchain-community

In [ ]:
# Downloading local LLM (llama)
!wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf -O llama-2-7b-chat.Q4_K_M.gguf

--2025-09-10 10:25:28--  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 3.166.152.65, 3.166.152.110, 3.166.152.105, ...
Connecting to huggingface.co (huggingface.co)|3.166.152.65|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/b0/ca/b0cae82fd4b3a362cab01d17953c45edac67d1c2dfb9fbb9e69c80c32dc2012e/08a5566d61d7cb6b420c3e4387a39e0078e1f2fe5f055f3a03887385304d4bfa?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.Q4_K_M.gguf%3B+filename%3D%22llama-2-7b-chat.Q4_K_M.gguf%22%3B&Expires=1757503528&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1NzUwMzUyOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9iMC9jYS9iMGNhZTgyZmQ0YjNhMzYyY2FiMDFkMTc5NTNjNDVlZGFjNjdkMWMyZGZiOWZiYjllNjljODBjMzJkYzIwMTJlLzA4YTU1NjZkNjFkN2NiNmI0MjBjM2U0Mzg3YTM5ZTAwNzhlMWYyZmU1ZjA1NWYzYTAzODg3Mzg

In [ ]:
# Setup ChromaDB to store skills and resume information about user locally.
# Embeddings to convert text to vectors and chromadb to store these vectors.
# Purpose is to store meaning of sentence and retrieve text (info) based upon meaning.

from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# Embeddings model (AI model to make numerical vectors from text)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Initialize ChromaDB collection
vectorstore = Chroma(collection_name="personal_data", embedding_function=embeddings)

# Example resume data to store in chromaDB
docs = [
    "Skilled in Python, Machine Learning, and Data Analysis.",
    "Worked on projects involving LangChain, Streamlit, and AI automation.",
    "Experience in writing professional cold emails for clients."
]

# Store into vector DB
vectorstore.add_texts(docs)
print("ChromaDB setup complete.")

ChromaDB setup complete.


In [ ]:
# Load Local LLM model
from langchain.llms import LlamaCpp

llm = LlamaCpp(
    model_path="llama-2-7b-chat.Q4_K_M.gguf",  # local LLM Model
    temperature=0.7, # Creativity of model
    max_tokens=400, # Length of generated email
    n_ctx=2048 # Max number of tokens remembered by model during email generation
)

print("Llama model loaded.")

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 

Llama model loaded.


CPU : SSE3 = 1 | SSSE3 = 1 | AVX = 1 | AVX2 = 1 | F16C = 1 | FMA = 1 | BMI2 = 1 | LLAMAFILE = 1 | OPENMP = 1 | REPACK = 1 | 
Model metadata: {'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '2', 'general.architecture': 'llama', 'llama.context_length': '4096', 'general.name': 'LLaMA v2', 'llama.embedding_length': '4096', 'llama.feed_forward_length': '11008', 'llama.attention.layer_norm_rms_epsilon': '0.000001', 'llama.rope.dimension_count': '128', 'llama.attention.head_count': '32', 'tokenizer.ggml.bos_token_id': '1', 'llama.block_count': '32', 'llama.attention.head_count_kv': '32', 'general.quantization_version': '2', 'tokenizer.ggml.model': 'llama', 'general.file_type': '15'}
Using fallback chat format: llama-2


In [ ]:
# Define Prompt template and chain=> user input + chromaDB + LLM model
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Prompt template
template = """
You are an expert email writer.
Write a professional cold email for a job application.

Job Title: {job_title}
Company: {company}
Relevant Skills: {skills}

Also include relevant achievements from my background:
{resume_data}

Email:
"""

prompt = PromptTemplate(
    input_variables=["job_title", "company", "skills", "resume_data"],
    template=template,
)

chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
# Testing our code to check all components work fine with random user input
job_title = "Machine Learning Intern"
company = "Google"
skills = "Python, Data Science, Deep Learning"

# Get relavent resume data from ChromaDB
docs = vectorstore.similarity_search(skills, k=2)
resume_data = " ".join([d.page_content for d in docs])

# Run the chain
email = chain.run(job_title=job_title, company=company, skills=skills, resume_data=resume_data)

print("Generated Cold Email:\n")
print(email)

Llama.generate: 26 prefix-match hit, remaining 61 prompt tokens to eval
llama_perf_context_print:        load time =   39591.48 ms
llama_perf_context_print: prompt eval time =   27126.45 ms /    61 tokens (  444.70 ms per token,     2.25 tokens per second)
llama_perf_context_print:        eval time =  229688.37 ms /   299 runs   (  768.19 ms per token,     1.30 tokens per second)
llama_perf_context_print:       total time =  257333.64 ms /   360 tokens
llama_perf_context_print:    graphs reused =        293


Generated Cold Email:

Subject: Applying for Machine Learning Internship at Google
Dear Hiring Manager,
I am writing to express my strong interest in the Machine Learning Intern position at Google. As a highly motivated and skilled data scientist with a passion for machine learning, I believe I would be an excellent fit for this role.
With a background in computer science and a proven track record of developing innovative machine learning solutions, I am confident in my ability to make meaningful contributions to the Google team. My skills in Python, Data Science, and Deep Learning are particularly relevant to this position, and I have a strong portfolio of projects that showcase my expertise in these areas.
In my current role as a data scientist at ABC Company, I have honed my skills in data analysis and machine learning, developing predictive models that have driven significant business outcomes. My experience working with cross-functional teams has also given me the ability to effec

In [ ]:
# Another test with different user input
job_title = "web developer Intern"
company = "microsoft"
skills = "html, css, js"

# Get resume data from ChromaDB
docs = vectorstore.similarity_search(skills, k=2)
resume_data = " ".join([d.page_content for d in docs])

# Run the chain
email = chain.run(job_title=job_title, company=company, skills=skills, resume_data=resume_data)

print("Generated Cold Email:\n")
print(email)

Llama.generate: 26 prefix-match hit, remaining 57 prompt tokens to eval
llama_perf_context_print:        load time =   39591.48 ms
llama_perf_context_print: prompt eval time =   24207.31 ms /    57 tokens (  424.69 ms per token,     2.35 tokens per second)
llama_perf_context_print:        eval time =  199965.72 ms /   263 runs   (  760.33 ms per token,     1.32 tokens per second)
llama_perf_context_print:       total time =  224610.29 ms /   320 tokens
llama_perf_context_print:    graphs reused =        260


Generated Cold Email:


Subject: Application for Web Developer Internship at Microsoft

Dear Hiring Manager,

I am writing to express my strong interest in the Web Developer Intern position at Microsoft. As a recent graduate with a degree in Computer Science, I am eager to apply my skills and knowledge in a dynamic and innovative environment like Microsoft.
With a solid background in HTML, CSS, and JavaScript, I am confident in my ability to contribute to the development of cutting-edge web applications. In addition, I have experience in writing professional cold emails for clients, which has honed my communication and problem-solving skills.
Throughout my academic career, I have consistently demonstrated a passion for programming and a commitment to excellence in all of my projects. I am particularly drawn to Microsoft's culture of innovation and creativity, and I believe that this internship would provide an ideal opportunity for me to learn from the best and make meaningful contribu